Q2-3

Import

In [161]:
import pandas as pd
import numpy as  np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import csv
import warnings

Data import

In [162]:
labelencoder = LabelEncoder()
pd.set_option ( "display.max_rows", 20000 )
pd.set_option ( "display.max_columns", 20 )
usecols_data = ['satisfaction_level','last_evaluation','number_project','average_montly_hours','time_spend_company','Work_accident','promotion_last_5years','sales','salary']
df_data_all = pd.read_csv( r'HW2_data/HW2_hr-analytics_train.csv' )
print ( "Number of NaN in the Data" )
print ( df_data_all.isna().sum() )
df_data = pd.read_csv( r'HW2_data/HW2_hr-analytics_train.csv' , usecols = usecols_data )
df_label = pd.read_csv( r'HW2_data/HW2_hr-analytics_train.csv' , usecols = ['left'] )
df_test = pd.read_csv( r'HW2_data/HW2_hr-analytics_test.csv' )

Number of NaN in the Data
satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
promotion_last_5years    0
sales                    0
salary                   0
left                     0
dtype: int64


各個特徵與離職率的關係探討

In [163]:
df_data_all_drop = df_data_all.select_dtypes(include = ["int64", "float64" ])
correlation_matrix = df_data_all_drop.corr()
correlation_with_target = correlation_matrix['left']
print(correlation_with_target.abs().sort_values(ascending=False))

left                     1.000000
satisfaction_level       0.389721
Work_accident            0.158327
time_spend_company       0.138764
average_montly_hours     0.069911
promotion_last_5years    0.064861
number_project           0.023949
last_evaluation          0.008456
Name: left, dtype: float64


捨棄非數值資料

In [164]:
df_data_drop = df_data.select_dtypes(include = ["int64", "float64" ])
df_test_drop = df_test.select_dtypes(include = ["int64", "float64" ])
df_data_nmp=df_data_drop.to_numpy()
data_splitted_arr_1 , data_splitted_arr_2 = np.vsplit(df_data_nmp, 2)
df_label_nmp=df_label.to_numpy()
label_splitted_arr_1 , label_splitted_arr_2 = np.vsplit(df_label_nmp, 2)
label_splitted_arr_1 = label_splitted_arr_1.ravel()
label_splitted_arr_2 = label_splitted_arr_2.ravel()

Train Model Without Encode 

In [165]:
warnings.filterwarnings("ignore", message="X has feature names, but LogisticRegression was fitted without feature names")
model = LogisticRegression( max_iter = 500 )

model.fit(data_splitted_arr_1, label_splitted_arr_1)
predictions = model.predict(data_splitted_arr_2)

model.score(data_splitted_arr_1,label_splitted_arr_1)
print ( "Before Encode" ) 
print("Accuracy:" ,metrics.accuracy_score(label_splitted_arr_2, predictions))
print("Precision:",metrics.precision_score(label_splitted_arr_2, predictions))
print("Recall:" ,metrics.recall_score(label_splitted_arr_2, predictions))

coef_model = list(model.coef_)
pos_coef = list()
for i in range(len(coef_model[0])):
    if coef_model[0][i] > 0:
        pos_coef.append ( i )

matrix = confusion_matrix(label_splitted_arr_2,predictions)
print ( "Confusion Matrix" )
print ( matrix )

Before Encode
Accuracy: 0.7068
Precision: 0.302734375
Recall: 0.2919020715630885
Confusion Matrix
[[3224  714]
 [ 752  310]]


Data PreProcessing and Encode

In [166]:
df_data = pd.get_dummies(df_data)
df_test = pd.get_dummies(df_test)

#df_data = (df_data-df_data.mean())/(df_data.std())
#['sales'] = labelencoder.fit_transform ( df_data['sales'] )
#df_data['salary'] = labelencoder.fit_transform ( df_data['salary'] )
#df_test['sales'] = labelencoder.fit_transform ( df_test['sales'] )
#df_test['salary'] = labelencoder.fit_transform ( df_test['salary'] )

df_data_nmp=df_data.to_numpy()
data_splitted_arr_1 , data_splitted_arr_2 = np.vsplit(df_data_nmp, 2)
df_label_nmp=df_label.to_numpy()
label_splitted_arr_1 , label_splitted_arr_2 = np.vsplit(df_label_nmp, 2)
label_splitted_arr_1 = label_splitted_arr_1.ravel()
label_splitted_arr_2 = label_splitted_arr_2.ravel()

Train Model

In [167]:
warnings.filterwarnings("ignore", message="X has feature names, but LogisticRegression was fitted without feature names")
model = LogisticRegression( max_iter = 500 )

model.fit(data_splitted_arr_1, label_splitted_arr_1)
predictions = model.predict(data_splitted_arr_2)

model.score(data_splitted_arr_1,label_splitted_arr_1)
print ( "After Encode" ) 
print("Accuracy:" ,metrics.accuracy_score(label_splitted_arr_2, predictions))
print("Precision:",metrics.precision_score(label_splitted_arr_2, predictions))
print("Recall:" ,metrics.recall_score(label_splitted_arr_2, predictions))

coef_model = list(model.coef_)
pos_coef = list()
for i in range(len(coef_model[0])):
    if coef_model[0][i] > 0:
        pos_coef.append ( i )

matrix = confusion_matrix(label_splitted_arr_2,predictions)
print ( "Confusion Matrix" )
print ( matrix )


After Encode
Accuracy: 0.75
Precision: 0.4219269102990033
Recall: 0.4783427495291902
Confusion Matrix
[[3242  696]
 [ 554  508]]


In [168]:
test_predictions = model.predict(df_test)
with open('HW2_hr-analytics_test_sol.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(["left"])
  for i in np.nditer(test_predictions):
      writer.writerow([i])